In [27]:
import datasets
import openai
import pandas as pd
import time
import os
from datasets import Dataset
from openai import OpenAI
from tqdm import tqdm

In [38]:
# hf_token = os.environ.get("HUGGING_FACE_TOKEN")
# openai_api_key = os.environ.get("OPENAI_API_KEY")

client = OpenAI(api_key='')

In [39]:
evaluation_data  = datasets.load_dataset("israel/JOPUjJHxWmI5x", use_auth_token='')

/Users/israela/Documents/repos/final/afri-rlhf/.conda/lib/python3.8/site-packages/datasets/load.py:2483: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


In [40]:
BASE_PROMPT = """Below is an interaction between a human and an AI fluent in English and Amharic, providing reliable and informative answers. The AI is supposed to answer test questions from the human with short responses saying just the answer and nothing else.

Human: {instruction}

 {input_text}

Assistant [Amharic] : """

In [41]:
def query_gpt4(instruction, input_text):
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
        {"role": "user", "content": f"{instruction}\n{input_text}"}
        ]
    )
    return response.choices[0].message.content.strip()

In [42]:
def save_to_csv(data, filename):
    df = pd.DataFrame(data, columns=['instruction', 'input', 'output', 'datasource', 'response'])
    df.to_csv(filename, index=False)

In [50]:
def evaluate(test_data, data_sources, output_dir='model_evaluation', sleep_duration=0, resume=False):
    os.makedirs(output_dir, exist_ok=True)
    data = []
    start_index = 0
    output_filename = os.path.join(output_dir, "gpt4_responses.csv")
    
    
    if resume and os.path.exists(output_filename):
        df = pd.read_csv(output_filename)
        start_index = len(df)
        data.extend(df.values.tolist())
        print(f"Resuming from Test Case {start_index + 1}")
        
    datasource_data = test_data.filter(lambda example: example['datasource'] in data_sources)
    
    resume_data = datasource_data[start_index:]   
    
    for i, (
        instruction,
        input_text,
        output_text,
        data_source
    ) in enumerate(
        tqdm(
            zip(
                resume_data['instruction'],
                resume_data['input'],
                resume_data['output'],
                resume_data['datasource']
            ),
            total=len(resume_data['input'])
        )
    ): 
  
        try:
            response = query_gpt4(instruction, input_text)
            data.append([instruction, input_text, output_text, data_source, response])
            save_to_csv(data, output_filename)
        except Exception as e:
            print(f"API Error for {data_source} - Test Case {start_index + i + 1}: {str(e)}")
            break
    
        time.sleep(sleep_duration)  # Add a sleep to avoid API rate limits
    
    
    print(f"Saved responses to {output_filename}")


In [51]:
# sample_test = Dataset.from_dict(evaluation_data['test'][:2])

In [45]:
evaluation_data['test'].unique('datasource')

['afrisent',
 'masakhanews',
 'masakhaner',
 'xlsum_summerization',
 'xlsum_reverse_summerization',
 'amharic_title_generation',
 'amharic_poem_completion',
 'amharic_poem_generation',
 'amharic_zefen_generation',
 'amharic_story_generation',
 'amharic_mezmur_completion',
 'amharic_mezmur_generation',
 'amharic_mt amh-eng',
 'amharic_mt eng-amh']

In [46]:
df = evaluation_data['test'].to_pandas()

df[df.datasource=="amharic_story_generation"]

,instruction,input,output,prompt_header,datasource
9541,ተረት ንገረኝ,\nላባ ያበቀለው ሰው\n,በወርቁ ደበሌ የተተረከ በአንድ ወቅት ሁለት ወንድ ልጆች የነበሩት ሰው ...,"Below is an instruction that describes a task,...",amharic_story_generation
9542,ተረት ንገረኝ,\nድመቶችና አይጦች\n,በመርጋ ደበሎ የተተረከ ድሮ፣ድሮ ድመቶችና አይጦች በህብረት አብረው ይኖ...,"Below is an instruction that describes a task,...",amharic_story_generation
9543,ተረት ንገረኝ,\nየአባትየው ምክር\n,በሞቱ ወዬሣ የተተረከ በአንድ ወቅት ባልና ሚስት አብረው ሲኖሩ ሚስትየው...,"Below is an instruction that describes a task,...",amharic_story_generation
9544,ተረት ንገረኝ,\nሞኙ ሌባ ውሻ\n,ተራኪው የማይታወቅ ከእለታት አንድ ቀን አንድ ውሻ ከጎረቤት ስጋ ሰርቆ ...,"Below is an instruction that describes a task,...",amharic_story_generation
9545,ተረት ንገረኝ,\nየኃይለሥላሴ ምህረት\n,በወርቁ መርሻ የተተረከ ሞጣ ከምትባል አንዲት መንደር ውስጥ የሚኖር ሰው...,"Below is an instruction that describes a task,...",amharic_story_generation
9546,ተረት ንገረኝ,\nጅብና ቀበሮ (2)\n,በዩሱፍ አደም ማንደሬ የተተረከ አንድ ቀን ጅብና ቀበሮ አብረው እያደኑ ...,"Below is an instruction that describes a task,...",amharic_story_generation
9547,ተረት ንገረኝ,\nደግለራሱና ክፉለራሱ\n,በሺርጉማ ሰራጋ የተተረከ በአንድ ወቅት ደግለራሱና ክፉለራሱ አብረው ጉ...,"Below is an instruction that describes a task,...",amharic_story_generation
9548,ተረት ንገረኝ,\nየመነኩሴው ምርቃት\n,ተራኪው የማይታወቅ አንድ ወንድ ልጅ የነበራቸው ባልና ሚስት ነበሩ፡፡ እ...,"Below is an instruction that describes a task,...",amharic_story_generation
9549,ተረት ንገረኝ,\nተኩላ፣ቀበሮና ጥንቸል\n,በአበበ ከበደ የተተረከ በአንድ ወቅት ተኩላ፣ ቀበሮና ጥንቸል አብረው ይ...,"Below is an instruction that describes a task,...",amharic_story_generation
9550,ተረት ንገረኝ,\nተንኮለኛው ቀበሮ\n,በመሃመድ አህመድ አልጋኒ የተተረከ በአንድ ወቅት ላም ያላት ሰጎንና በሬ...,"Below is an instruction that describes a task,...",amharic_story_generation


In [47]:
evaluate(evaluation_data['test'],['amharic_story_generation'],output_dir="amharic_story_generation", resume=False)

100%|██████████| 25/25 [1:20:46<00:00, 193.87s/it]

Saved responses to amharic_story_generation/gpt4_responses.csv


In [48]:
response_df = pd.read_csv('amharic_story_generation/gpt4_responses.csv')

In [49]:
response_df

,instruction,input,output,datasource,response
0,ተረት ንገረኝ,\nላባ ያበቀለው ሰው\n,በወርቁ ደበሌ የተተረከ በአንድ ወቅት ሁለት ወንድ ልጆች የነበሩት ሰው ...,amharic_story_generation,ቀድሞ በአንድ አይነት ገንዘብ ሀብተኛ በሚመረት ገኛ ላይ አንድ ወጣት ይኖ...
1,ተረት ንገረኝ,\nድመቶችና አይጦች\n,በመርጋ ደበሎ የተተረከ ድሮ፣ድሮ ድመቶችና አይጦች በህብረት አብረው ይኖ...,amharic_story_generation,"በመካከለኛ ግብረ-ኣዳራሽ, መንገድ መስመር የተሰነዘሩበት አረራ ማእከል, ..."
2,ተረት ንገረኝ,\nየአባትየው ምክር\n,በሞቱ ወዬሣ የተተረከ በአንድ ወቅት ባልና ሚስት አብረው ሲኖሩ ሚስትየው...,amharic_story_generation,"ባንድ ጊዜ ጊዜ አምላክ እንዲህ አለው፡ ""ልጄ ሁሉም እነዚህ ለማድረግ የሚ..."
3,ተረት ንገረኝ,\nሞኙ ሌባ ውሻ\n,ተራኪው የማይታወቅ ከእለታት አንድ ቀን አንድ ውሻ ከጎረቤት ስጋ ሰርቆ ...,amharic_story_generation,ባዶ ከነበረው ፈረሰኛ ስር በራሪ ሲሳል ነበር፡፡ በሩ በሮማኖቹ እያሰረጠጠ...
4,ተረት ንገረኝ,\nየኃይለሥላሴ ምህረት\n,በወርቁ መርሻ የተተረከ ሞጣ ከምትባል አንዲት መንደር ውስጥ የሚኖር ሰው...,amharic_story_generation,ይሆድ በመካከለኛው ዓረብ ሀገር ውስጥ ለሚገኘው የኃይሌ ንጉሥ አለቃነት ኃ...
5,ተረት ንገረኝ,\nጅብና ቀበሮ (2)\n,በዩሱፍ አደም ማንደሬ የተተረከ አንድ ቀን ጅብና ቀበሮ አብረው እያደኑ ...,amharic_story_generation,ቀድሞ ትረፍ ላይ ጅብ ነበረ። ጅባል ያሉ ጨጓራቹ ለካቲት ተፈጨ። ሴት ጅብ...
6,ተረት ንገረኝ,\nደግለራሱና ክፉለራሱ\n,በሺርጉማ ሰራጋ የተተረከ በአንድ ወቅት ደግለራሱና ክፉለራሱ አብረው ጉ...,amharic_story_generation,በጥሩ ጊዜ አንድ ጎበኛ ፍሬዎችን በማስረጃው በሽታው የሚፈጥራን ካስፈለስና...
7,ተረት ንገረኝ,\nየመነኩሴው ምርቃት\n,ተራኪው የማይታወቅ አንድ ወንድ ልጅ የነበራቸው ባልና ሚስት ነበሩ፡፡ እ...,amharic_story_generation,ቀድሞ በትኩስ ጊዜ በአንድ መነኩሴ አገር ላይ ታላቅ አንድ ንጉሥ ነበረ። ...
8,ተረት ንገረኝ,\nተኩላ፣ቀበሮና ጥንቸል\n,በአበበ ከበደ የተተረከ በአንድ ወቅት ተኩላ፣ ቀበሮና ጥንቸል አብረው ይ...,amharic_story_generation,በባህር ዳርቻ መናገሻ ላይ ተኩላ፣ ቀበሮና ጥንቸል ነበሩ። በአንድነት ብለ...
9,ተረት ንገረኝ,\nተንኮለኛው ቀበሮ\n,በመሃመድ አህመድ አልጋኒ የተተረከ በአንድ ወቅት ላም ያላት ሰጎንና በሬ...,amharic_story_generation,"ተንኮለኛው ቀበሮ ትምህርት የሌባው የተመራቁ ልጆችን የገለፀው ነው- ""ስል..."
